In [1]:
from cataclop.core.models import *

In [2]:
from cataclop.pmu.models import *

In [ ]:
races = Race.objects.filter(start_at__date="2018-10-01")

for race in races:
    print(race, race.declared_player_count, race.player_set.count())

In [6]:
players = Player.objects.filter(final_odds_ref_unibet__isnull=False, position=1)

for p in players:
    print(p.race, p.position, p.final_odds, p.final_odds_ref, p.final_odds_unibet, p.final_odds_ref_unibet)

2019-02-05 R1C1 13:45 1 7.1 6.8 6.4 5.7
2019-02-05 R1C2 14:22 1 19.3 19.6 18.3 18.9
2019-02-05 R1C3 14:57 1 6.9 9.7 6.2 8.8
2019-02-05 R1C4 15:30 1 2.9 4.4 3.0 5.2
2019-02-05 R1C5 16:05 1 2.3 2.2 2.3 4.2
2019-02-05 R1C6 16:40 1 14.8 18.3 12.7 16.7
2019-02-05 R1C7 17:15 1 2.9 6.6 3.0 3.3
2019-02-05 R1C8 17:50 1 4.4 5.4 3.6 4.0
2019-02-05 R2C1 11:55 1 3.4 3.2 2.9 2.7
2019-02-05 R2C2 12:25 1 2.9 4.7 2.6 3.4
2019-02-05 R2C3 12:57 1 4.9 5.8 3.9 6.6
2019-02-05 R2C4 13:25 1 1.7 1.6 1.7 1.5
2019-02-05 R2C5 14:04 1 1.4 1.6 1.6 1.2
2019-02-05 R2C6 14:37 1 7.1 13.3 7.6 8.5
2019-02-05 R2C7 15:12 1 7.7 10.8 7.9 11.5
2019-02-05 R2C8 15:47 1 6.6 6.2 5.3 5.5
2019-02-05 R3C1 12:10 1 2.1 2.4 2.6 2.8
2019-02-05 R3C2 12:40 1 9.1 18.4 7.3 9.8
2019-02-05 R3C3 13:10 1 87.9 26.3 98.8 26.6
2019-02-05 R4C1 16:22 1 2.8 2.7 2.6 2.7
2019-02-05 R4C2 16:57 1 4.4 6.0 4.5 7.9
2019-02-05 R4C3 17:32 1 16.0 31.3 18.3 17.0
2019-02-05 R4C4 18:07 1 3.3 3.4 2.8 5.1
2019-02-05 R4C5 18:39 1 3.0 3.6 2.7 2.5
2019-02-05 R4C6 19:0

In [ ]:
bets = Bet.objects.filter(player__isnull=False).order_by('-created_at')

for bet in bets:
    print( '{} {} / {}: bet {}€ on {}'.format(bet.created_at, bet.url, bet.program, bet.amount, bet.player.num))

In [ ]:
horse = Horse.objects.filter(name__icontains='what secret').first()
horse

In [ ]:
player = Player.objects.filter(horse=horse, num=3).last()
player.race.id

In [ ]:
odds = Odds.objects.filter(player=player)
odds

In [ ]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from cataclop.ml import preprocessing
from cataclop.ml import exploration

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

from cataclop.ml.pipeline import factories

In [ ]:
dataset = factories.Dataset.factory('2019-01-07', params={
    'race_id':60962
})

In [ ]:
dataset.load()

In [ ]:
dataset.players[['num', 'horse_id', 'final_odds_ref', 'position']]